# Dependencies

This application requires the following dependencies:
- opencv-python 4.5.3.56
- numpy 1.21.6
- pandas 1.3.5
- tensorflow 2.9.1
- tensorflow-hub 0.12.0

In [ ]:
!pip install opencv-python==4.5.3.56
!pip install numpy==1.21.6
!pip install pandas==1.3.5
!pip install tensorflow==2.9.1
!pip install tensorflow-hub==0.12.0

#### Set **use_camera_image = True** to use a camera image

#### Set **use_camera_image = False** to use a predefined image (sample_image.png in this directory):
<img src="sample_image.png" />

Sample image obtained from https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1

In [ ]:
use_camera_image = True

In [ ]:
from arena_api import enums
from arena_api.buffer import BufferFactory
from arena_api.system import system

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow_hub as hub
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
# This loads the model from the TensorFlow website
# Labels are loaded locally (labels.csv included with this example)
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")

# Loading csv with labels of classes
# csv file from https://github.com/gabrielcassimiro17/raspberry-pi-tensorflow/blob/main/labels.csv
labels = pd.read_csv('labels.csv',sep=';',index_col='ID')
labels = labels['OBJECT (2017 REL.)']

# TensorFlow score threshold

In [ ]:
score_threshold = 0.4

### The following variables define the display window width and height

In [ ]:
window_width = 800
window_height = 600

### The following function waits for the user to select a device

In [ ]:
def select_device_from_user_input():
    """
    This function waits for the user to select a device
    an exception
    """

    device_infos = None
    selected_index = None
    while selected_index is None:
        device_infos = system.device_infos
        if len(device_infos) == 0:
            print("No camera connected\nPress enter to search again")
            input()
            continue
        print("Devices found:")
        for i in range(len(device_infos)):
            print(f"\t{i}. {device_infos[i]['model']} SN: {device_infos[i]['serial']}")

        while True:
            line = input("Selection: ")
            try:
                selected_index = int(line)
                if 0 <= selected_index < len(device_infos):
                    break
                else:
                    print(f"Please enter a valid number between 0 and {len(device_infos)-1}\n")
            except Exception as e:
                print("\nPlease enter a valid number\n")

    selected_model = device_infos[selected_index]['model']
    print(f"\nCreate device: {selected_model}...")
    device = system.create_device(device_infos=device_infos[selected_index])[0]
    
    return device

### The following function sets up and runs the camera

In [ ]:
def run_camera():

    # create devices
    device = select_device_from_user_input()
    
    device.tl_stream_nodemap.get_node('StreamBufferHandlingMode').value = 'NewestOnly'
    device.tl_stream_nodemap.get_node('StreamPacketResendEnable').value = True
    device.tl_stream_nodemap.get_node('StreamAutoNegotiatePacketSize').value = True
    
    isp_bayer_pattern = device.nodemap.get_node('IspBayerPattern').value
    is_color_camera = False
    
    if isp_bayer_pattern != 'NONE':
        is_color_camera = True
    
    if is_color_camera == True:
        device.nodemap.get_node('PixelFormat').value = "BayerRG8"
    else:
        device.nodemap.get_node('PixelFormat').value = "Mono8"
        
    # grab images -------------------------------------------------------------
    get_image_buffers(device, is_color_camera)

    # clean up ----------------------------------------------------------------

    # This function call with no arguments will destroy all of the
    # created devices. Having this call here is optional, if it is not
    # here it will be called automatically when the system module is unloading.
    system.destroy_device()
    print('Destroyed all created devices')

In [1]:
def process_image(display_img):
    
    input_img = cv2.resize(display_img,(window_width, window_height))
    
    # Convert image to RGB for TensorFlow
    rgb_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)

    rgb_tensor = tf.convert_to_tensor(rgb_img, dtype=tf.uint8)

    #Add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor , 0)

    boxes, scores, classes, num_detections = detector(rgb_tensor)

    pred_labels = classes.numpy().astype('int')[0]

    pred_labels = [labels[i] for i in pred_labels]
    pred_boxes = boxes.numpy()[0].astype('int')
    pred_scores = scores.numpy()[0]

    #loop throughout the detections and place a box around it  
    for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < score_threshold:
            continue

        score_txt = f'{100 * round(score,0)}'
        output_img = cv2.rectangle(rgb_img,(xmin, ymax),(xmax, ymin),(0,255,0),1)      
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(output_img, label,(xmin, ymax-10), font, 1, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(output_img, score_txt,(xmax, ymax-10), font, 1, (255,255,255), 1, cv2.LINE_AA)
        display_img = cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR)
        
    return display_img

In [ ]:
def get_image_buffers(device, is_color_camera=False):
    
    key = -1
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    
    device.start_stream()
    
    while True:
        image_buffer = device.get_buffer()  # optional args
        nparray = np.ctypeslib.as_array(image_buffer.pdata,shape=(image_buffer.height, image_buffer.width, int(image_buffer.bits_per_pixel / 8))).reshape(image_buffer.height, image_buffer.width, int(image_buffer.bits_per_pixel / 8))

        if is_color_camera == True:
            #converted_img = BufferFactory.convert(image_buffer, enums.PixelFormat.BGR8)
            #display_img = np.ctypeslib.as_array(converted_img.pdata,shape=(converted_img.height, converted_img.width, int(converted_img.bits_per_pixel / 8))).reshape(converted_img.height, converted_img.width, int(converted_img.bits_per_pixel / 8))
            display_img = cv2.cvtColor(nparray, cv2.COLOR_BayerBG2BGR)
            nparray = cv2.cvtColor(display_img, cv2.COLOR_BGR2GRAY)
        else:
            display_img = cv2.cvtColor(nparray, cv2.COLOR_GRAY2BGR)
            
        processed_image = process_image(display_img)
            
        cv2.resizeWindow("Image", window_width, window_height)
        cv2.imshow("Image", processed_image)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        #BufferFactory.destroy(converted_image)
        device.requeue_buffer(image_buffer)

    cv2.destroyAllWindows()
    device.stop_stream()

In [ ]:
def read_sample_image():
    
    key = -1
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    
    display_img = cv2.imread("sample_image.png", cv2.IMREAD_COLOR)
    
    processed_image = process_image(display_img)
    
    while True:
        cv2.resizeWindow("Image", window_width, window_height)
        cv2.imshow("Image", processed_image)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
            
    cv2.destroyAllWindows()

In [ ]:
if __name__ == '__main__':
    try:
        print('WARNING:\nTHIS EXAMPLE MIGHT CHANGE THE DEVICE(S) SETTINGS!')
        print('Example started')
        
        if use_camera_image == True:
            run_camera()
        else:
            read_sample_image()
        
        print('Example finished successfully')
    except BaseException as be:
        print(be)
        raise be